# Extract data from invitrodb 

In [51]:
from sqlalchemy import create_engine, inspect, text

In [52]:
engine = create_engine("mysql+pymysql://root@localhost/prod_internal_invitrodb_v4_1")

inspector = inspect(engine)
table_names = inspector.get_table_names()

In [ ]:
# mc5_param has hitcalls and AC50
# cytotox has median and lower bound cytotoxicity estimates
# chemical links chid to dtxsid, which I can use to map to OASIS compounds

In [14]:
engine = create_engine("mysql+pymysql://root@localhost/prod_internal_invitrodb_v4_1")

with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM mc5 LIMIT 10"))
    
    column_names = result.keys()
    print(column_names)
    
    rows = result.fetchall()
    for row in rows:
        print(row)

RMKeyView(['m5id', 'm4id', 'aeid', 'modl', 'hitc', 'fitc', 'coff', 'actp', 'model_type', 'created_date', 'modified_date', 'modified_by'])
(1097403, 1205858, 2658, 'exp2', 0.8003413127151334, 1, 24.90768, None, 2, datetime.datetime(2022, 9, 29, 10, 15, 19), datetime.datetime(2022, 9, 29, 10, 15, 19), 'MFeshuk.MFeshuk.MFeshuk')
(1097404, 1205859, 2658, 'poly1', 0.0013580605917213062, 1, 24.90768, None, 2, datetime.datetime(2022, 9, 29, 10, 15, 19), datetime.datetime(2022, 9, 29, 10, 15, 19), 'MFeshuk.MFeshuk.MFeshuk')
(1097405, 1205860, 2658, 'poly1', 0.025622175506414175, 1, 24.90768, None, 2, datetime.datetime(2022, 9, 29, 10, 15, 19), datetime.datetime(2022, 9, 29, 10, 15, 19), 'MFeshuk.MFeshuk.MFeshuk')
(1097406, 1205861, 2659, 'poly1', 0.05612543762487714, 1, 20.0, None, 2, datetime.datetime(2022, 9, 29, 10, 15, 19), datetime.datetime(2022, 9, 29, 10, 15, 19), 'MFeshuk.MFeshuk.MFeshuk')
(1097407, 1205862, 2659, 'poly1', 5.0455715889335306e-06, 1, 20.0, None, 2, datetime.datetime(202

In [55]:
# This is where I can get AC50 values and hitcalls
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM mc5_param WHERE hit_param IN ('ac50', 'hitcall') LIMIT 10;"))
    
    column_names = result.keys()
    print(column_names)
    
    rows = result.fetchall()
    for row in rows:
        print(row)

RMKeyView(['m5id', 'aeid', 'hit_param', 'hit_val'])
(4382439, 2, 'hitcall', 2.4704463935303977e-08)
(4382439, 2, 'ac50', 100.00000000000001)
(4382440, 2, 'hitcall', 3.1319497092249218e-12)
(4382440, 2, 'ac50', 100.00000000000001)
(4382441, 2, 'hitcall', 6.803483156160703e-13)
(4382441, 2, 'ac50', 100.00000000000001)
(4382442, 2, 'hitcall', 0.9478650415589537)
(4382442, 2, 'ac50', 0.29460089536501555)
(4382443, 2, 'hitcall', 0.0)
(4382444, 2, 'hitcall', 0.0)


In [27]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM cytotox LIMIT 5"))
    
    column_names = result.keys()
    print(column_names)
    
    rows = result.fetchall()
    for row in rows:
        print(row)

# cytotox_median = median AC50 for cytotoxicty burst endpoints
# cytotox_lower_bnd = cytotox_median - 3*MAD

# chid = chemical id? 
# so, cytotoxicity is computed once for each chemical, which means that it doesn't consider sensitivity of specific cell types
# has anyone analyzed the cytotoxicity across different cell types? 


RMKeyView(['chid', 'cytotox_median_raw', 'cytotox_mad', 'global_mad', 'cytotox_median_log', 'cytotox_median_um', 'cytotox_lower_bound_um', 'ntested', 'nhit', 'cytotox_lower_bound_log', 'created_date'])
(20005, None, None, 0.15970696427871311, 3.0, 1000.0, 1000.0, 71.0, 0.0, 3.0, datetime.datetime(2023, 5, 2, 15, 3, 9))
(20007, None, None, 0.15970696427871311, 3.0, 1000.0, 1000.0, 67.0, 0.0, 3.0, datetime.datetime(2023, 5, 2, 15, 3, 9))
(20009, None, None, 0.15970696427871311, 3.0, 1000.0, 1000.0, 67.0, 0.0, 3.0, datetime.datetime(2023, 5, 2, 15, 3, 9))
(20014, None, None, 0.15970696427871311, 3.0, 1000.0, 1000.0, 70.0, 0.0, 3.0, datetime.datetime(2023, 5, 2, 15, 3, 9))
(20015, 0.7166615083847401, 0.20407883207397942, 0.15970696427871311, 0.7166615083847401, 5.207886472165505, 1.7279875756572285, 67.0, 5.0, 0.23754061554860073, datetime.datetime(2023, 5, 2, 15, 3, 9))


In [28]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM chemical LIMIT 5"))
    
    column_names = result.keys()
    print(column_names)
    
    rows = result.fetchall()
    for row in rows:
        print(row)

# compounds have DTXSID, so I can merge with OASIS based on this

RMKeyView(['chid', 'casn', 'chnm', 'dsstox_substance_id'])
(20005, '60-35-5', 'Acetamide', 'DTXSID7020005')
(20006, '103-90-2', 'Acetaminophen', 'DTXSID2020006')
(20007, '968-81-0', 'Acetohexamide', 'DTXSID7020007')
(20009, '75-05-8', 'Acetonitrile', 'DTXSID7020009')
(20014, '520-45-6', 'Dehydroacetic acid', 'DTXSID6020014')


In [ ]:
# assay (aid) has timepoint, organism, tissue, cell, cell-free characteristics, etc
# assay component (acid) lists all of the different types of measurements (components) from the same assay (ie. cell growth kinetics and viability)
# example query: "SELECT * FROM assay_component WHERE aid = '1' LIMIT 5"
# assay component endpoint (aeid) has intended target info, signal_direction, whether it's a positive control or a burst_assay or a cell_viability_assay
# gene has gene ids and names
# intended_target has the intended target of each aeid, and maps to target ids

# Q: what do 'burst assays' all include? Does it still make sense to analyze those on their own? What about cell viability assays? Can we assume that anything not a burst or viability assay is a more specific target?


Path forward:
- Get list of DTXSID from OASIS list
- Query all toxcast chemicals from this list (get list of chid)
- Get all assays endpoints (aeid) for these chid
- Get assay endpoint description, plus key assay info, for all endpoints (cell, tissue, etc)
- Get cytotoxicity info for OASIS chid
- Get mc5_param of interest: AC50, bmd, bmdu, bmdl, and hit call

Still to do:
- Decide what assays to treat as "specific" endpoints and which to lump into cell viability
- Figure out if there are many of the same viability assays run for the same compound in same cell types: what to do here?